In [529]:
#model definition

import random
import time
from collections import deque
import numpy as np

import torch as tc
import torch.nn as nn
import torch.optim as opt
import torch.nn.init as init
DEVICE = tc.device("cuda" if tc.cuda.is_available() else "cpu")

import gym
import gym_snake
env = gym.make('snake-v0')

class Option(object):
    HUNGRY_RATE=20
    ROW=8
    COL=8

class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.in_ch=2
        self.out_ch=3
        
        self.c1=nn.Sequential(
            nn.Conv2d(self.in_ch,3,3,padding=1,padding_mode="circular"),
            nn.LeakyReLU(0.1),
            
            nn.Conv2d(3,self.out_ch,3,padding=1,padding_mode="circular"),
        )
        
        self.d1=nn.Sequential(
            nn.Linear(self.out_ch*Option.ROW*Option.COL,512),
            nn.LeakyReLU(0.1),
            
            nn.Linear(512,256),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            
            nn.Linear(256,3),
        )
        for m in [self.c1,self.d1]:
            if type(m)==nn.Linear or type(m)==nn.Conv2d:
                init.xavier_uniform_(m.weight.data)
                init.xavier_uniform_(m.bias.data)
                
    def forward(self,x):
        x=x.to(DEVICE)
        x=x.reshape((-1,2,Option.ROW,Option.COL))
        x=self.c1(x)
        x=x.reshape(-1,self.out_ch*Option.ROW*Option.COL)
        return self.d1(x)

In [972]:
#load
net_trained=NN().cuda()
net_trained.load_state_dict(tc.load('./netw.pt'))

<All keys matched successfully>

In [984]:
import time

net_trained.eval()

with tc.no_grad():
    obs=env.reset()
    while True:
        res=net_trained(tc.tensor(obs).to(DEVICE)).detach().squeeze().tolist()
        obs, rwd, done, _ = env.step(res.index(max(res)))
        env.render()
        time.sleep(0.05)
        if done:
            break
    env.close()